## 3. Dependency parser splitting (EXPERIMENTING)

Once the typo's have been corrected, the next step is to delimit the text. Delimiting involves the splitting of a sumation of words through characters such as <i>',', ';'</i> but also by specific words such as <i>'and'</i>, <i>'or'</i> and <i>'but'</i> for example. 

### 3.1 Import the required libraries

In [1]:
import spacy
from spacy.matcher import Matcher
import pandas as pd
import numpy as np

### 3.2 Load in the typo-corrected and translated data

In [ ]:
# Determine the location of the dataframe containing the typo-corrected text
file_location = "../data/example_data/output/1k_subset/translated_products_1k.parquet"

# Read the dataframe
translated_and_typo_corrected_df = pd.read_parquet(file_location)

# Display the dataframe
display(translated_and_typo_corrected_df)

,products_id,products_and_services,language (ISO-code),typo_corrected,translated_text
0,164399edbf8e880dc2e856f50d51e720bd0a8abe,"fish, frozen and deep-frozen",en,"fish, frozen and deep-frozen","fish, frozen and deep-frozen"
1,b0d3c55743b1b858ec2843c8870116bb8af543fd,drilling and test boring - equipment,en,drilling and test boring - equipment,drilling and test boring - equipment
2,b14c038972e6a52bfbf3ffbe77def57a62c5b9cf,well-management services,nl,well-management services,well-management services
3,abadc2542b4b5c1ecfe41c22afb2347b1d9b65af,electronic data processing - software,en,electronic data processing - software,electronic data processing - software
4,60c58ad2ef34d96fae028f1039fab03dec9eb9a2,communication,it,communication,communication
...,...,...,...,...,...
995,ecfa87628d5e5249db7730bf2bed09f8c4419dc9,roofing materials,en,looking materials,roofing materials
996,1e15564c0be3b13bb11516b0499fe75d8d7f976b,"energy conservation, consultants",en,"energy conservation, consultant","energy conservation, consultants"
997,eb4f959622941f4dd9b40355669cba97858ba66b,infrared heaters,en,inflamed theaters,infrared heaters
998,08ea29fcbe3c2f14f62b96e5af2f7e838fff0773,bakery and confectionery industry - machinery ...,en,baker and confectioner industry - machinery an...,bakery and confectionery industry - machinery ...


### 3.3 

In [ ]:
# create a list of cases in how the noun-adjective pairs can be constructed
# the list is used to create a pattern for the matcher
# - noun and adjectives could be directly next to each other as such <adj><noun>: "Warm milk"
# - noun and adjectives could be separated by a verb as such <adj><verb><noun>: "Warm up milk"
# - noun and adjectives could be separated by a preposition as such <adj><prep><noun>: "Warm with milk"

In [ ]:
for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_)

manufacture NOUN NN ROOT
of ADP IN prep
retail ADJ JJ amod
space NOUN NN compound
equipment NOUN NN pobj
, PUNCT , punct
discounted VERB VBN amod
shop NOUN NN compound
furniture NOUN NN appos


In [ ]:
for sent in doc.sents:
    print(sent)

manufacture of retail space equipment, discounted shop furniture


In [3]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("manufacture of retail space equipment, discounted shop furniture and shop shelvings")

# Extract noun-adjective pairs
noun_adjective_pairs = []
for token in doc:
    if token.pos_ == "NOUN":
        # Look for the adjective attached to the noun
        adjective = [child.text for child in token.children if child.pos_ == "ADJ"]
        if adjective:
            noun_adjective_pairs.append((token.text, adjective[0]))

In [7]:
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("manufacturer of retail space equipment, discounted shop furniture and shop shelvings")
displacy.render(doc, style="dep")

In [8]:
print(noun_adjective_pairs)

[('equipment', 'retail')]


### 3.4 Export the dataframe containing the delimited text

In [ ]:
# Define the path for the new dataframe
output_path_delimiter_corrected = "../data/example_data/output/delimited_and_corrected_products.parquet"

# Write the new dataframe to the path
df_split.write.parquet(output_path_delimiter_corrected)